In [1]:
#imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split as tts

In [2]:
#Variables
TRAIN_THRESHOLD = 0.8
DROPNAN = False

In [3]:
training = pd.read_csv('./ressources/training/training.csv')
#On enleve les NaN
if DROPNAN:
    training = training.dropna()
else:
    training.fillna(method = 'ffill',inplace = True)

In [4]:
X = training.Image.values
del training['Image']
Y = training.values
print(Y)

[[66.03356391 39.00227368 30.22700752 ... 72.93545865 43.13070677
  84.48577444]
 [64.33293617 34.9700766  29.9492766  ... 70.26655319 45.46791489
  85.48017021]
 [65.05705263 34.90964211 30.90378947 ... 70.19178947 47.27494737
  78.65936842]
 ...
 [66.69073171 36.84522146 31.66641951 ... 75.96359236 49.46257171
  78.11712   ]
 [70.96508235 39.85366588 30.54328471 ... 75.96359236 50.06518588
  79.58644706]
 [66.93831111 43.42450963 31.09605926 ... 75.96359236 45.90048
  82.7730963 ]]


In [5]:
x = []
for i in tqdm(X):
    q = [int(j) for j in i.split()]
    x.append(q)
len(x)

x = np.array(x)
x = x.reshape(X.shape[0], 96,96,1)
x  = x/255.0
x.shape

100%|██████████| 7049/7049 [00:11<00:00, 616.58it/s]


(7049, 96, 96, 1)

In [6]:
x_train,x_test,y_train,y_test = tts(x,Y,random_state = 69,test_size = 1-TRAIN_THRESHOLD)
x_train.shape

(5639, 96, 96, 1)

In [7]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [8]:
tf.test.is_gpu_available(cuda_only=True)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [9]:
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(96, 96, 1)))

headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.4)(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(len(y_train[0]), activation="softmax")(headModel)

for layer in baseModel.layers:
	layer.trainable = False

model = Model(inputs=baseModel.input, outputs=headModel)
model.compile(optimizer = 'adam',loss = 'mean_squared_error', metrics = ['mae','acc'])
model.summary()

ValueError: Cannot assign to variable block1_conv1/kernel:0 due to variable shape (3, 3, 1, 64) and value shape (64, 3, 3, 3) are incompatible

In [ ]:

model.fit(x_train,y_train,batch_size=256, epochs=50,validation_data=(x_test,y_test))
model.save_weights("./checkpoints/CNN1.ckpt")
